In [63]:
import pandas as pd
import glob

In [ ]:
# --- Parámetros ---
carpeta_tablas = "vcf_tables/*.txt"  # patrón de archivos
var_interes_pos = 114787  # posición de tu variante de interés (ej: chr1:123456)
var_interes_chr = "chr1"

# --- Función para procesar una tabla ---
def variantes_cis(archivo):
    df = pd.read_csv(archivo, sep="\t", header=None,
                     names=["chr","pos","GT","PS"])
    df = df[df["chr"] == var_interes_chr].copy()

    # Buscar la variante de interés
    var_int = df[df["pos"] == str(var_interes_pos)]
    if var_int.empty:
        return pd.DataFrame(), archivo  # Variante no encontrada

    # Sacar haplotipo y PS de la variante de interés
    gt = var_int["GT"].iloc[0]
    ps = var_int["PS"].iloc[0]
    hap_int = gt.split("|")[0] if gt.startswith("1|") or gt.startswith("0|") else None
    # hap_int = alelo en hap1 (puedes cambiar a hap2 si quieres)

    # Filtrar variantes en el mismo PS y mismo haplotipo
    same_ps = df[df["PS"] == ps].copy()
    same_ps["hap1"], same_ps["hap2"] = zip(*same_ps["GT"].str.split("|"))
    same_ps = same_ps[same_ps["hap1"] == hap_int]  # compara haplotipo 1
    return same_ps, archivo

In [66]:
# --- Procesar todas las tablas ---
resultados = []
for archivo in glob.glob(carpeta_tablas):
    same_ps, nombre = variantes_cis(archivo)
    if not same_ps.empty:
        same_ps["muestra"] = nombre
        resultados.append(same_ps)

# --- Combinar resultados ---
if resultados:
    df_all = pd.concat(resultados, ignore_index=True)
    #print("Variantes que segregan con", var_interes_chr, var_interes_pos)
    #print(df_all)

    # Conteo global de co-segregación
    conteo = df_all.groupby(["chr","pos"]).size().reset_index(name="n_muestras")
    conteo = conteo.sort_values("n_muestras", ascending=False)
    print("\nConteo global:")
    print(conteo)
else:
    print("No se encontraron variantes co-segregantes con la variante de interés.")


Conteo global:
    chr     pos  n_muestras
5  chr1  198629           3
2  chr1  114787           3
1  chr1  111698           2
0  chr1  109783           2
3  chr1  133910           1
4  chr1  136436           1


las variantes que más me interesan son las que están en 